This part of the pipeline reannotates all genomes that did not fail the checkM QC using Bakta at default settings.

### Checking dependencies

In [ ]:
conda activate bakta
bakta --version
conda deactivate

### Paths and parameters

#### Pipeline input folders

In [ ]:
refseq_genomes="./00-refseq/data"
failed_checkm="./00-refseq/failed_checkm"

#### Pipeline output folders

In [ ]:
task_root="./01-bakta"
bakta_wdir="$task_root/workdir"
bakta_genomes="$task_root/genomes"
bakta_gffs="$task_root/gffs"
bakta_proteomes="$task_root/proteomes"
bakta_proteomes_gbk="$task_root/proteomes_gbk"

mkdir -p $task_root $bakta_wdir $bakta_genomes $bakta_gffs $bakta_proteomes $bakta_proteomes_gbk

#### Tool pointers and parameters

In [ ]:
bakta_db="$HOME/bin/bakta"

n_pipes=3
n_cores=22
n_cores_per_pipe=$(( n_cores / n_pipes ))

### Genome reannotation with Bakta

In [ ]:
conda activate bakta

Only reannotate the ones that did not fail the checkM QC.

In [ ]:
comm -23 <(dir -1 $refseq_genomes | xargs basename -s .fna | sort) <(cat $failed_checkm | sort) | \
parallel --eta -j $n_pipes -I % \
bakta -t $n_cores_per_pipe -o $bakta_wdir/% -d $bakta_db --prefix % --locus-tag % --translation-table 11 \
$refseq_genomes/%.fna

In [ ]:
conda deactivate

Removing all non-ASCII characters to avoid errors downstream the pipeline

In [ ]:
dir -1 */* | grep -v .png | xargs -I % bash -c 'strings -w -s "" -1 % > %+clean'
dir -1 */* | grep -v .png | grep clean | cut -d '+' -f 1 | xargs -I % mv %+clean %

The command below should then return empty.

In [ ]:
LC_ALL=C grep -n -P [$'\x80'-$'\xFF'] */*

Move the nucleotide and amino acid fastas, the GFF annotation table and the annotated Genbank file to the results folders.

In [ ]:
dir -1 $bakta_wdir | xargs -I % cp -u $bakta_wdir/%/%.faa $bakta_proteomes/%.faa
dir -1 $bakta_wdir | xargs -I % cp -u $bakta_wdir/%/%.gbff $bakta_proteomes_gbk/%.gbff
dir -1 $bakta_wdir | xargs -I % cp -u $bakta_wdir/%/%.gff3 $bakta_gffs/%.gff
dir -1 $bakta_wdir | xargs -I % cp -u $bakta_wdir/%/%.fna $bakta_genomes/%.fna

In [ ]:
dir -1 $refseq_genomes | wc -l
wc -l $failed_checkm
dir -1 $bakta_wdir | wc -l